In [4]:
# required imports for the function here
# some of these might be extraneous since they're just being grabbed from the total pipeline
# but right now, we just need to make some catalogs!
%matplotlib inline

import numpy as np
from numpy.lib.recfunctions import append_fields
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import halotools.mock_observables as mo
import halotools.sim_manager as sm
import astropy.coordinates as coord

In [2]:
fname = './l0250_d70b.catalog' # file name to z0.0.catalog
mthresh = 9e11  # threshold mass in Msun/h
gnewton = 4.302e-6
lbox = 250.0
vhost_min = 240.0
vrat_frac = 0.3
vsub_min = vhost_min * vrat_frac
nrand = 10

# define dict pointing to all marks of interest
rs_dict = {'halo_id':(0,'i8'), 'halo_mass':(2,'f8'), 'halo_vmax':(3,'f8'), 'halo_rvir':(5,'f8'),
           'halo_rs':(6,'f8'), 'halo_x':(8,'f8'), 'halo_y':(9,'f8'), 'halo_z':(10,'f8'),
           'halo_spin':(17,'f8'), 'halo_ctoa':(28, 'f8'), 'halo_pid':(33,'i8')}

reader = sm.TabularAsciiReader(fname, rs_dict, row_cut_min_dict={'halo_mass':mthresh},
                               row_cut_eq_dict={'halo_pid':-1})
hosts_data_master = reader.read_ascii()

fname = './l0250_d200b.catalog' # file name to z0.0.catalog
mthresh = 7e11  # threshold mass in Msun/h
gnewton = 4.302e-6
lbox = 250.0
vhost_min = 240.0
vrat_frac = 0.3
vsub_min = vhost_min * vrat_frac
nrand = 10

# define dict pointing to all marks of interest
rs_dict = {'halo_id':(0,'i8'), 'halo_mass':(2,'f8'), 'halo_vmax':(3,'f8'), 'halo_rvir':(5,'f8'),
           'halo_rs':(6,'f8'), 'halo_x':(8,'f8'), 'halo_y':(9,'f8'), 'halo_z':(10,'f8'),
           'halo_spin':(17,'f8'), 'halo_ctoa':(28, 'f8'), 'halo_pid':(33,'i8')}

reader = sm.TabularAsciiReader(fname, rs_dict, row_cut_min_dict={'halo_mass':mthresh},
                               row_cut_eq_dict={'halo_pid':-1})
hosts_data_matching = reader.read_ascii()


...Processing ASCII data of file: 
/data/asv13/projects/haloenviron-pipeline/l0250_d70b.catalog
 
Total number of rows in detected data = 2535889
Number of rows in detected header = 15 

... working on chunk 0 of 1
Total runtime to read in ASCII = 12.93 seconds



...Processing ASCII data of file: 
/data/asv13/projects/haloenviron-pipeline/l0250_d200b.catalog
 
Total number of rows in detected data = 2342337
Number of rows in detected header = 15 

... working on chunk 0 of 1
Total runtime to read in ASCII = 12.65 seconds




In [5]:
c_master = coord.SkyCoord(x=hosts_data_master['halo_x'], y=hosts_data_master['halo_y'], z=hosts_data_master['halo_z'],
                         unit='Mpc', frame='icrs', representation='cartesian')

In [7]:
c_matching = coord.SkyCoord(x=hosts_data_matching['halo_x'], y=hosts_data_matching['halo_y'], z=hosts_data_matching['halo_z'],
                            unit='Mpc', frame='icrs', representation='cartesian')

In [8]:
idx, sep2d, sep3d = coord.match_coordinates_3d(c_matching, c_master)

In [40]:
np.percentile(sep3d,70)

0.00260249879924997

In [52]:
mask = (sep3d.value <= hosts_data_matching['halo_rvir']*.001*.1)

In [53]:
mask

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [54]:
print len(mask), np.sum(mask)

76261 55746


In [55]:
matched_catalog = hosts_data_matching[np.where(mask==True)]

In [56]:
len(matched_catalog)

55746